In [1]:
pip install psycopg2-binary sentence-transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 13.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 27.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 29.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 MB 16.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 25.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 11.0 MB/s eta 0:

In [2]:
import psycopg2
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# PostgreSQL connection settings
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "postgres",
    "port": 5432,
}

In [4]:
# Load Sentence Transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("SELECT review_id, review FROM public.reviews")
    reviews = cur.fetchall()

    for review_id, review_text in reviews:
        embedding = model.encode(review_text).tolist()

        # Update the table with generated embedding
        cur.execute(
            "UPDATE public.reviews SET review_embedding = %s WHERE review_id = %s;",
            (embedding, review_id),
        )
        conn.commit()
        print("Embeddings updated successfully.")
    
finally:
    cur.close()
    if conn is not None:
        conn.close()

Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.
Embeddings updated successfully.


In [5]:
def find_similar_reviews(query_text, top_n=5):
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    query_embedding = model.encode(query_text).tolist()

    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Cosine similarity search
    cursor.execute(
        """
        SELECT customer_name, review, (review_embedding <=> %s::vector) AS similarity
        FROM public.reviews
        ORDER BY similarity ASC
        LIMIT %s;
        """,
        (query_embedding, top_n),
    )

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    print("\nSimilar Reviews:")
    for customer_name, review, similarity in results:
        print(f"{customer_name} {review} (Similarity: {similarity:.4f})")

In [6]:
find_similar_reviews("The customer service was very helpful and kind.",4)


Similar Reviews:
Finn Miller Positive experience dealing with customer support. They were knowledgeable and able to assist me with my product inquiry efficiently. (Similarity: 0.3276)
Eve Williams Very impressed with the customer service! They were quick to address my concerns and offered a satisfactory solution. (Similarity: 0.3523)
Charlie Brown Helpful and friendly staff. They were patient and understanding when I was having trouble navigating the website. (Similarity: 0.3984)
Alice Smith Amazing customer service! They went above and beyond to help me with my order. I will definitely be back. (Similarity: 0.4560)


In [7]:
find_similar_reviews("Very impressed with the customer service! They were quick to address my concerns and offered a satisfactory solution.",4)


Similar Reviews:
Eve Williams Very impressed with the customer service! They were quick to address my concerns and offered a satisfactory solution. (Similarity: 0.0000)
Alice Smith Amazing customer service! They went above and beyond to help me with my order. I will definitely be back. (Similarity: 0.3641)
Finn Miller Positive experience dealing with customer support. They were knowledgeable and able to assist me with my product inquiry efficiently. (Similarity: 0.4105)
Harry Rodriguez Terrible customer service! I waited for hours for a reply to a question, and no one replied yet. (Similarity: 0.4217)
